In [1]:
import src.view.pnl_view as pnlview
from multiprocessing import Pool, Process, Manager, cpu_count
from src.core.logger import Logger
from src.core.stopwatch import Stopwatch
from src.core.stopwatch_logger import StopwatchLogger
from src.synthetic_book_merger import SyntheticBookMerger
from src.synthetic_security_loader import SyntheticSecurityLoader
import src.sniper_opp_loader
from src.sniper_opp_loader import SniperOppLoader

from src.view import data_view as dview
from src.view.opp_view import OppView, to_opp_view, display_opp_view
from src.view.cell_view import CellView
from src.view.color import Color, Colors, WebColors
from src.pcap_location_params import PCapLocationParams
from src.core.postgres_connection import PostgresConnection
from src.view.cell_view import CellView
from src.market_date import *

import pandas as pd
import numpy as np
import sqlite3
import time
import matplotlib.pyplot as plt
import os.path

# pandarallel.initialize(progress_bar=True, use_memory_fs=True)
pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220
pd.options.display.max_rows = 263

In [2]:
import importlib
import src.view.opp_view
import src.view.data_view as dview
import src.core.sort_utils as sort_utils
import src.view.edge_summary_view
import src.synthetic_security_loader
import src.synthetic_book_merger
importlib.reload(module=src.view.opp_view)
importlib.reload(module=dview)
importlib.reload(module=src.view.edge_summary_view)
importlib.reload(module=src.synthetic_security_loader)
importlib.reload(module=src.synthetic_book_merger)
from src.view.opp_view import OppView
from src.view.edge_summary_view import get_edge_summary
from src.synthetic_security_loader import SyntheticSecurityLoader
from src.synthetic_book_merger import SyntheticBookMerger

In [3]:
synth_secs = SyntheticSecurityLoader(340).result().set_index('id')
market_dates = {pid: set(get_feather_dates(340, name = f'synthetic_book_{pid}', month=[5])) for pid in range(1, 12)}
market_dates = list(set.intersection(*[set(v) for k, v in market_dates.items()]))
symbols=[
    '+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',
    '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',
    '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1']

opp_loader = SniperOppLoader(340, dates=market_dates, log_level=1)
opp_root = opp_loader.load_asset_opp_root('zs')
opp = opp_loader.load('zs')

[Info ] [2021-07-23 23:40:36.065308795]  |  Begin loading asset zs (11 symbols, 21 dates)
[Info ] [2021-07-23 23:40:39.396059902]  |  End loading asset zs (11 symbols, 21 dates).  Took 0 days 00:00:03.330610889


In [5]:
synth_bk_date_groupby = opp_loader.synth_bk().groupby(level=[0])
odg = opp.set_index(opp.index.set_names(['market_date', 'symbol', 'opp_start_eid'])).groupby(level=[0])
opp_w_bk_root = SyntheticBookMerger(odg, synth_bk_date_groupby).merge(parallel=True)

[Info ] [2021-07-23 23:41:13.997673604]  |  Begin loading synth books (all on 2021-05-17, 2021-05-23, 2021-05-19, 2021-05-25, and 17 others)
[Info ] [2021-07-23 23:41:14.225735660]  |  [2021-05-19] Processing 2021-05-19 polygon 1[Info ] [2021-07-23 23:41:14.225625782]  |  [2021-05-23] Processing 2021-05-23 polygon 1[Info ] [2021-07-23 23:41:14.225833731]  |  [2021-05-25] Processing 2021-05-25 polygon 1[Info ] [2021-07-23 23:41:14.225553152]  |  [2021-05-17] Processing 2021-05-17 polygon 1[Info ] [2021-07-23 23:41:14.226216023]  |  [2021-05-13] Processing 2021-05-13 polygon 1[Info ] [2021-07-23 23:41:14.226038238]  |  [2021-05-31] Processing 2021-05-31 polygon 1[Info ] [2021-07-23 23:41:14.226393604]  |  [2021-05-04] Processing 2021-05-04 polygon 1[Info ] [2021-07-23 23:41:14.226915432]  |  [2021-05-18] Processing 2021-05-18 polygon 1[Info ] [2021-07-23 23:41:14.226566111]  |  [2021-05-20] Processing 2021-05-20 polygon 1[Info ] [2021-07-23 23:41:14.227120106]  |  [2021-05-05] Processing

In [6]:
opp_bk_tmp = opp_w_bk_root.copy()
opp_bk_tmp['t_tod'] = opp_bk_tmp['t_time'] - opp_bk_tmp.index.get_level_values('market_date') - pd.Timedelta('1d')
opp_bk_tmp = dview.with_groupby_origin_col(opp_bk_tmp, levels=['market_date', 'symbol', 'opp_start_eid'])
ov_tmp = to_opp_view(opp_bk_tmp)
display_opp_view(ov_tmp)

In [7]:
opp_bk= opp_w_bk_root.copy()
opp_bk = pd.merge(
    opp_bk.reset_index('symbol').rename(columns={'symbol': 'sym'}),
    synth_secs, left_on='sym', right_on='symbol', how='left')\
    .set_index(opp_bk.index)

opp_bk_edge_summ = get_edge_summary(opp_bk, cpp=50.0, fee='member106j_fee')
es = opp_bk_edge_summ.copy()
edge_col_names = [(f'{e * .25:.2f}', dview.fmt_float_lambda(ndec=0)) for e in range(-2, 5)]
sdict = dict([(c, dview.fmt_float_lambda(ndec=2)) for c in ['net', 'net_cash', 'fill_edge', 'net_fill_cash']] + edge_col_names)
es = es.reset_index().set_index(['market_date', 'symbol'])
es.style.format(sdict)

In [8]:
es['month'] = pd.DatetimeIndex(es.index.get_level_values('market_date')).month
es_syms = es.index.get_level_values('symbol')
es['sym'] = es_syms

def get_monthly_pnl(x):
    return pd.DataFrame(data={str(m): [x.loc[x.name,m]['net_fill_cash'].sum() if (x.name, m) in x.index else 0.0] for m in range(4,8)})

es[['net_fill_cash', 'month', 'sym']].groupby(['sym', 'month']).sum().groupby(level='sym').apply(get_monthly_pnl)

,,4,5,6,7
sym,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),0,0.0,488.26,0.0,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),0,0.0,82.58,0.0,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),0,0.0,525.52,0.0,0.0
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),0,0.0,-1061.78,0.0,0.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),0,0.0,-733.92,0.0,0.0
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,0,0.0,-473.36,0.0,0.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0,0.0,-218.92,0.0,0.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,0,0.0,-733.62,0.0,0.0
+ZSN1 -(ZSN1-ZSX1) -ZSX1,0,0.0,-254.20,0.0,0.0
